In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.metrics import confusion_matrix,roc_curve,auc
from sklearn.metrics import max_error,mean_squared_error
import torch
from torch import nn
import torch.utils.data as Data
from torch.nn import init
from torch.nn import functional as F
import torch.optim as optim
import seaborn as sns
import os

##  1. Data processing

In [ ]:
data=pd.read_csv('./data.csv')
data_feature=data[['K+', 'Na+', 'Ca2+', 'Mg2+', 'Cl-', 'SO42-','HCO3-', 'SiO2']]

data_target=data.temp
data_feature.head()

# 添加特征：商
list1=['K+', 'Na+', 'Ca2+', 'Mg2+', 'Cl-', 'SO42-','HCO3-', 'SiO2']

for i in range(0,len(list1)+1):
    for j in range(i+1,len(list1)):
        if i!=3:
            data_feature[list1[j]+'/'+list1[i]]=data[list1[j]]/data[list1[i]]
        if j!=3:
            data_feature[list1[i]+'/'+list1[j]]=data[list1[i]]/data[list1[j]]
    

data_feature.columns

# 添加特征：次方,对数
for i in range(0,len(list1)):

    if i!=3:
        data_feature['ln'+list1[i]]=data[list1[i]].apply(lambda x: math.log(x))
data_feature.columns
# data_feature.describe()

data_join=pd.concat([data_feature,data_target],axis=1)

data_corr=pd.DataFrame(data_join.corr().temp)
data_corr.index
list3=['K+', 'Na+', 'Ca2+', 'Mg2+', 'Cl-', 'SO42-', 'HCO3-', 'SiO2', 'Na+/K+',
       'K+/Na+', 'Ca2+/K+', 'K+/Ca2+', 'Mg2+/K+', 'Cl-/K+', 'K+/Cl-',
       'SO42-/K+', 'K+/SO42-', 'HCO3-/K+', 'K+/HCO3-', 'SiO2/K+', 'K+/SiO2',
       'Ca2+/Na+', 'Na+/Ca2+', 'Mg2+/Na+', 'Cl-/Na+', 'Na+/Cl-', 'SO42-/Na+',
       'Na+/SO42-', 'HCO3-/Na+', 'Na+/HCO3-', 'SiO2/Na+', 'Na+/SiO2',
       'Mg2+/Ca2+', 'Cl-/Ca2+', 'Ca2+/Cl-', 'SO42-/Ca2+', 'Ca2+/SO42-',
       'HCO3-/Ca2+', 'Ca2+/HCO3-', 'SiO2/Ca2+', 'Ca2+/SiO2', 'Mg2+/Cl-',
       'Mg2+/SO42-', 'Mg2+/HCO3-', 'Mg2+/SiO2', 'SO42-/Cl-', 'Cl-/SO42-',
       'HCO3-/Cl-', 'Cl-/HCO3-', 'SiO2/Cl-', 'Cl-/SiO2', 'HCO3-/SO42-',
       'SO42-/HCO3-', 'SiO2/SO42-', 'SO42-/SiO2', 'SiO2/HCO3-', 'HCO3-/SiO2',
       'lnK+', 'lnNa+', 'lnCa2+', 'lnCl-', 'lnSO42-', 'lnHCO3-', 'lnSiO2']
a=0
for i in range(0,len(list3)):
#     print(i)
    if abs(data_corr[list3[i]:list3[i]]['temp'].item())<0.3:
        a=a+1
#         print(i)
        del data_feature[list3[i]]
        

data_join=pd.concat([data_feature,data_target],axis=1)

data_corr=pd.DataFrame(data_join.corr().temp)
# data_corr.to_excel('data_corr.xlsx','Sheet1',index=True)

## 2. Net training

In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(data_feature,data_target,train_size=0.8,random_state=100)

X_test_f=X_test
y_test_f=y_test
X_train, X_test, y_train, y_test =  train_test_split(X_train,y_train,train_size=0.9,random_state=100)

In [ ]:
#train 
X_train = torch.tensor(np.array(X_train), dtype=torch.float)
y_train=torch.tensor(np.array(y_train), dtype=torch.float)
#valiation
X_validation = torch.tensor(np.array(X_test), dtype=torch.float)
y_validation=torch.tensor(np.array(y_test), dtype=torch.float)
#test
X_test=torch.tensor(np.array(X_test_f), dtype=torch.float)
y_test=torch.tensor(np.array(y_test_f), dtype=torch.float)


batch_size = 100 
train_dataset = Data.TensorDataset(X_train, y_train)
train_loader = Data.DataLoader(
    dataset=train_dataset,            
    batch_size=batch_size,      
    shuffle=True,               
                  
    )

validation_dataset = Data.TensorDataset(X_test, y_test)
validation_loader = Data.DataLoader(
    dataset=validation_dataset,            
    batch_size=batch_size,      
    shuffle=True,              
                  
    )

In [ ]:
## Net1
class Net(nn.Module):
    def __init__(self, n_feature):
        super(Net, self).__init__()      # call father function to init 
        self.input = nn.Linear(n_feature, 50)  # function prototype: `torch.nn.Linear(in_features, out_features, bias=True)
        self.relu=nn.PReLU()

        self.output  = nn.Linear(50,  1)
        self.dropout=nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.input(x)
        x=self.dropout(x)
        x = self.relu(x)

        x=self.output(x)
        x = x.squeeze(-1)
        return x
net = Net(35)  

In [ ]:
LR=0.01
EPOCH=40000
scores=0


font = {'family' : 'Times New Roman',
                    'weight' : 'normal',
                    'size'   : 15,
                    }
    

In [ ]:
# 模型评价函数
def Ev_score(x,y):
    return 1-((x-y).var())/(x.var())

#定义损失函数
loss = nn.MSELoss()    

#定义优化函数
optimizer = optim.Adamax(net.parameters(), lr=LR ,eps=0.00000001)# built-in random gradient descent function

In [ ]:
def train_model():
    net.train() 
    for X,y in train_loader:
        
        out = net(X)
        l = loss(out,y)
        
        
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        

In [ ]:
if __name__=='__main__':
    for epoch in range(0,EPOCH+1):
        train_model()
        
        
        net.eval() 
        
        validate_score=Ev_score(y_validation.detach().numpy(),net(X_validation).detach().numpy())
        train_score=Ev_score(y_train.detach().numpy(),net(X_train).detach().numpy())
        
        
        if validate_score>scores:
            
            scores=validate_score
#             torch.save(net.state_dict(), 'net1.pth')
#             torch.save( net,'net1.pkl')
            print('train_score:%f'%train_score)
            print('validate_score:%f'%validate_score)
            print('\n')


## 3. Result analysis


In [ ]:
model=torch.load('net1.pkl')

In [ ]:
def pie(net):
    validation_error=(net(X_validation )-y_validation).detach().numpy()
    test_error=(net(X_test )-y_test).detach().numpy()
    validation_error=pd.DataFrame(abs(validation_error))
    test_error=pd.DataFrame(abs(test_error))
    error=pd.concat([validation_error,test_error],axis=0)
    x=['0-1','1-10','10-20','>20']
    y=[int(error.loc[error[0]<1].count()),
       int(error.loc[(error[0]>1)&(error[0]<10)].count()),
       int(error.loc[(error[0]>10)&(error[0]<20)].count()),
       int(error.loc[error[0]>20].count())]
    plt.axes(aspect=1)
    plt.rcParams.update({'font.sans-serif' : 'Times New Roman',
                    'font.size': 12})
    plt.pie(y,autopct='%2.f%%',explode=[0,0,0,0],colors=['lime','yellow','orange','r'],shadow=0)
    plt.legend(x,loc=(0.9,0.6))

#     plt.savefig('./pie.jpg',dpi=500, bbox_inches='tight')
    plt.show()

In [ ]:
def fit(net):
    a=np.arange(0,350,2)
    b=a
    c=a+20
    d=a-20
    leg=['y = x','y=x+20','y=x-20','train ','validation ','test']
    plt.plot(a, b, lw=1)
    plt.plot(a, c, lw=1)
    plt.plot(a, d, lw=1)
    plt.scatter(y_train.detach().numpy(), net(X_train).detach().numpy(),c='b')
    plt.scatter(y_validation.detach().numpy(), net(X_validation).detach().numpy(),c='orange')
    plt.scatter(y_test.detach().numpy(), net(X_test).detach().numpy(),c='r')
        
        
    plt.xlim((0,300)) #设置坐标轴范围
    plt.ylim((0,300))

    x_ticks = np.arange(0, 350, 50)
    y_ticks = np.arange(0, 350, 50)
    plt.xticks(x_ticks)
    plt.yticks(y_ticks)
#     matplotlib.rcParams['font.sans-serif']=['simhei']
   
    
    plt.xlabel('x',font)
    plt.ylabel('y',font)
    
    plt.legend(leg,loc='best')
        
#     plt.savefig('fit1.jpg',dpi=500, bbox_inches='tight')
    plt.show()

In [ ]:
fit(model)
pie(model)